Notebook to plot and "graph network thingy" the cuts for acceptance vs energy to see why the acceptance is so much different for the 7 GeV energy beam compared to the 8, 9, and 10

In [1]:
import ROOT
import pandas as pd
from common_analysis_tools import *
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

Welcome to JupyROOT 6.24/04


In [2]:
os.nice(18)
ROOT.EnableImplicitMT()

ROOT.gStyle.SetOptStat(0)

In [3]:
beam_bin_filter = """
int get_beam_bin_index(double e_beam) {
        return static_cast<int>(e_beam-6.5) + 1;
}
"""

t_bin_filter = """
int get_t_bin_index(double t) {
    if (t <= 0.4) {
        return static_cast<int>(t/0.1)+1;
    }
    else if (t > 0.4 && t <= 0.9) {
        return static_cast<int>((t-0.4)/0.25)+5;
    }
    else if (t > 0.9 && t <= 1.9) {
        return static_cast<int>((t-0.9)/0.5)+7;
    }
    else {
        return -1;
    }
}
"""
ROOT.gInterpreter.Declare(t_bin_filter)
ROOT.gInterpreter.Declare(beam_bin_filter)


True

In [9]:

def build_recon_dataframe(channel):
    recon_file_and_tree_pipkmks_spring = get_flat_file_and_tree(f'{channel}', 'spring', 'phasespace', filtered=False)
    recon_file_and_tree_pipkmks_fall = get_flat_file_and_tree(f'{channel}', 'fall', 'phasespace', filtered=False)
    recon_file_and_tree_pipkmks_2017 = get_flat_file_and_tree(f'{channel}', '2017', 'phasespace', filtered=False)

    df = ROOT.RDataFrame(recon_file_and_tree_pipkmks_spring[1], [recon_file_and_tree_pipkmks_spring[0], recon_file_and_tree_pipkmks_fall[0], recon_file_and_tree_pipkmks_2017[0]])
    return df

#uglyyyyyyyyy code sorry
def define_columns(df, channel):
    if channel == 'pipkmks':
        return df.Define('p_pt', 'sqrt(p_px_measured*p_px_measured + p_py_measured*p_py_measured)').Define('p_p', 'sqrt(p_px*p_px + p_py*p_py + p_pz*p_pz)').Define('ks_px', "pip2_px + pim_px").Define('ks_py', "pip2_py + pim_py").Define('ks_pz', "pip2_pz + pim_pz").Define('ks_E', "pip2_E + pim_E").Define('ks_m', "sqrt(ks_E*ks_E - ks_px*ks_px - ks_py*ks_py - ks_pz*ks_pz)").Define('ppip_px', 'pip1_px + p_px').Define('ppip_py', 'pip1_py + p_py').Define('ppip_pz', 'pip1_pz + p_pz').Define('ppip_E', 'pip1_E + p_E').Define('ppip_m', 'sqrt(ppip_E*ppip_E - ppip_px*ppip_px - ppip_py*ppip_py - ppip_pz*ppip_pz)').Define('ks_px_measured', "pip2_px_measured + pim_px_measured").Define('ks_py_measured', "pip2_py_measured + pim_py_measured").Define('ks_pz_measured', "pip2_pz_measured + pim_pz_measured").Define('ks_E_measured', "pip2_E_measured + pim_E_measured").Define('ks_m_measured', "sqrt(ks_E_measured*ks_E_measured - ks_px_measured*ks_px_measured - ks_py_measured*ks_py_measured - ks_pz_measured*ks_pz_measured)").Define('mxpx_ppipkmks', '-p_px_measured - pip1_px_measured - km_px_measured - ks_px_measured').Define('mxpy_ppipkmks', '-p_py_measured - pip1_py_measured - km_py_measured - ks_py_measured').Define('mxpz_ppipkmks', 'e_beam - p_pz_measured - pip1_pz_measured - km_pz_measured - ks_pz_measured').Define('mxe_ppipkmks', 'e_beam + 0.938272088 - p_E_measured - pip1_E_measured - km_E_measured - ks_E_measured').Define('mx2_ppipkmks', 'mxe_ppipkmks*mxe_ppipkmks - mxpx_ppipkmks*mxpx_ppipkmks - mxpy_ppipkmks*mxpy_ppipkmks - mxpz_ppipkmks*mxpz_ppipkmks').Define('missing_px', '-p_px - pip1_px - ks_px - km_px').Define('missing_py', '-p_py - pip1_py - ks_py - km_py').Define('missing_pz', 'e_beam - p_pz - pip1_pz - ks_pz - km_pz').Define('missing_E', 'e_beam + 0.938 - p_E - pip1_E - ks_E - km_E').Define('kmp_px', 'p_px + km_px').Define('kmp_py', 'p_py + km_py').Define('kmp_pz', 'p_pz + km_pz').Define('kmp_E', 'p_E + km_E').Define('kmp_m', 'sqrt(kmp_E*kmp_E - kmp_px*kmp_px - kmp_py*kmp_py - kmp_pz*kmp_pz)').Define('missing_m', 'sqrt(missing_E*missing_E - missing_px*missing_px - missing_py*missing_py - missing_pz*missing_pz)').Define('kspip_px', 'pip1_px + ks_px').Define('kspip_py', 'pip1_py + ks_py').Define('kspip_pz', 'pip1_pz + ks_pz').Define('kspip_E', 'pip1_E + ks_E').Define('kspip_m', 'sqrt(kspip_E*kspip_E - kspip_px*kspip_px - kspip_py*kspip_py - kspip_pz*kspip_pz)').Define('kmpip_px', 'pip1_px + km_px').Define('kmpip_py', 'pip1_py + km_py').Define('kmpip_pz', 'pip1_pz + km_pz').Define('kmpip_E', 'pip1_E + km_E').Define('kmpip_m', 'sqrt(kmpip_E*kmpip_E - kmpip_px*kmpip_px - kmpip_py*kmpip_py - kmpip_pz*kmpip_pz)').Define('pipkmks_px', 'pip1_px + km_px + ks_px').Define('pipkmks_py', 'pip1_py + km_py + ks_py').Define('pipkmks_pz', 'pip1_pz + km_pz + ks_pz').Define('pipkmks_E', 'pip1_E + km_E + ks_E').Define('pipkmks_m', 'sqrt(pipkmks_E*pipkmks_E - pipkmks_px*pipkmks_px - pipkmks_py*pipkmks_py - pipkmks_pz*pipkmks_pz)').Define('kmks_px', 'km_px + ks_px').Define('kmks_py', 'km_py + ks_py').Define('kmks_pz', 'km_pz + ks_pz').Define('kmks_E', 'km_E + ks_E').Define('kmks_m', 'sqrt(kmks_E*kmks_E - kmks_px*kmks_px - kmks_py*kmks_py - kmks_pz*kmks_pz)').Define('e_bin', 'int(e_beam-6.5) +1').Define('t_bin', 'get_t_bin_index(mand_t)')
    elif channel == 'pimkpks':
        return df.Define('p_pt', 'sqrt(p_px_measured*p_px_measured + p_py_measured*p_py_measured)').Define('p_p', 'sqrt(p_px*p_px + p_py*p_py + p_pz*p_pz)').Define('ks_px', "pim2_px + pip_px").Define('ks_py', "pim2_py + pip_py").Define('ks_pz', "pim2_pz + pip_pz").Define('ks_E', "pim2_E + pip_E").Define('ks_m', "sqrt(ks_E*ks_E - ks_px*ks_px - ks_py*ks_py - ks_pz*ks_pz)").Define('ks_px_measured', "pim2_px_measured + pip_px_measured").Define('ks_py_measured', "pim2_py_measured + pip_py_measured").Define('ks_pz_measured', "pim2_pz_measured + pip_pz_measured").Define('ks_E_measured', "pim2_E_measured + pip_E_measured").Define('ks_m_measured', "sqrt(ks_E_measured*ks_E_measured - ks_px_measured*ks_px_measured - ks_py_measured*ks_py_measured - ks_pz_measured*ks_pz_measured)").Define('mxpx_ppimkpks', '-p_px_measured - pim1_px_measured - kp_px_measured - ks_px_measured').Define('mxpy_ppimkpks', '-p_py_measured - pim1_py_measured - kp_py_measured - ks_py_measured').Define('mxpz_ppimkpks', 'e_beam - p_pz_measured - pim1_pz_measured - kp_pz_measured - ks_pz_measured').Define('mxe_ppimkpks', 'e_beam + 0.938272088 - p_E_measured - pim1_E_measured - kp_E_measured - ks_E_measured').Define('mx2_ppimkpks', 'mxe_ppimkpks*mxe_ppimkpks - mxpx_ppimkpks*mxpx_ppimkpks - mxpy_ppimkpks*mxpy_ppimkpks - mxpz_ppimkpks*mxpz_ppimkpks').Define('ppim_px', 'pim1_px + p_px').Define('ppim_py', 'pim1_py + p_py').Define('ppim_pz', 'pim1_pz + p_pz').Define('ppim_E', 'pim1_E + p_E').Define('ppim_m', 'sqrt(ppim_E*ppim_E - ppim_px*ppim_px - ppim_py*ppim_py - ppim_pz*ppim_pz)').Define('missing_px', '-p_px - pim1_px - ks_px - kp_px').Define('missing_py', '-p_py - pim1_py - ks_py - kp_py').Define('missing_pz', 'e_beam - p_pz - pim1_pz - ks_pz - kp_pz').Define('missing_E', 'e_beam + 0.938 - p_E - pim1_E - ks_E - kp_E').Define('missing_m', 'sqrt(missing_E*missing_E - missing_px*missing_px - missing_py*missing_py - missing_pz*missing_pz)').Define('kpp_px', 'p_px + kp_px').Define('kpp_py', 'p_py + kp_py').Define('kpp_pz', 'p_pz + kp_pz').Define('kpp_E', 'p_E + kp_E').Define('kpp_m', 'sqrt(kpp_E*kpp_E - kpp_px*kpp_px - kpp_py*kpp_py - kpp_pz*kpp_pz)').Define('ksp_px', 'p_px + ks_px').Define('ksp_py', 'p_py + ks_py').Define('ksp_pz', 'p_pz + ks_pz').Define('ksp_E', 'p_E + ks_E').Define('ksp_m', 'sqrt(ksp_E*ksp_E - ksp_px*ksp_px - ksp_py*ksp_py - ksp_pz*ksp_pz)').Define('kspim_px', 'pim1_px + ks_px').Define('kspim_py', 'pim1_py + ks_py').Define('kspim_pz', 'pim1_pz + ks_pz').Define('kspim_E', 'pim1_E + ks_E').Define('kspim_m', 'sqrt(kspim_E*kspim_E - kspim_px*kspim_px - kspim_py*kspim_py - kspim_pz*kspim_pz)').Define('kppim_px', 'pim1_px + kp_px').Define('kppim_py', 'pim1_py + kp_py').Define('kppim_pz', 'pim1_pz + kp_pz').Define('kppim_E', 'pim1_E + kp_E').Define('kppim_m', 'sqrt(kppim_E*kppim_E - kppim_px*kppim_px - kppim_py*kppim_py - kppim_pz*kppim_pz)').Define('pimkpks_px', 'pim1_px + kp_px + ks_px').Define('pimkpks_py', 'pim1_py + kp_py + ks_py').Define('pimkpks_pz', 'pim1_pz + kp_pz + ks_pz').Define('pimkpks_E', 'pim1_E + kp_E + ks_E').Define('pimkpks_px_measured', "pim1_px_measured + kp_px_measured + ks_px_measured").Define('pimkpks_py_measured', "pim1_py_measured + kp_py_measured + ks_py_measured").Define('pimkpks_pz_measured', "pim1_pz_measured + kp_pz_measured + ks_pz_measured").Define('pimkpks_pt', 'sqrt(pimkpks_px_measured*pimkpks_px_measured + pimkpks_py_measured*pimkpks_py_measured)').Define('pimkpks_p_pt_diff', 'pimkpks_pt - p_pt').Define('pimkpks_m', 'sqrt(pimkpks_E*pimkpks_E - pimkpks_px*pimkpks_px - pimkpks_py*pimkpks_py - pimkpks_pz*pimkpks_pz)').Define('kpks_px', 'kp_px + ks_px').Define('kpks_py', 'kp_py + ks_py').Define('kpks_pz', 'kp_pz + ks_pz').Define('kpks_E', 'kp_E + ks_E').Define('kpks_m', 'sqrt(kpks_E*kpks_E - kpks_px*kpks_px - kpks_py*kpks_py - kpks_pz*kpks_pz)').Define('e_bin', 'get_beam_bin_index(e_beam)').Define('t_bin', 'get_t_bin_index(mand_t)').Define('ppip_px', 'p_px + pip_px').Define('ppip_py', 'p_py + pip_py').Define('ppip_pz', 'p_pz + pip_pz').Define('ppip_E', 'p_E + pip_E').Define('ppip_m', 'sqrt(ppip_E*ppip_E - ppip_px*ppip_px - ppip_py*ppip_py - ppip_pz*ppip_pz)')
    
def apply_filters_get_report(df, channel):
    if channel == 'pipkmks':
        delta_cut = PPIP_MASS_CUT
        lambda_cut = KMP_MASS_CUT
        missing_all_cut = MX2_PPIPKMKS_CUT
        kstar_cut = KSTAR_ALL_CUT_PIPKMKS
        f1_region = F1_SIGNAL_REGION_PIPKMKS
    elif channel == 'pimkpks':
        delta_cut = PPIM_MASS_CUT
        lambda_cut = KSP_MASS_CUT
        missing_all_cut = MX2_PPIMKPKS_CUT
        kstar_cut = KSTAR_ALL_CUT_PIMKPKS
        f1_region = F1_SIGNAL_REGION_PIMKPKS
    
    filtered_df = df.Filter(T_RANGE).Filter(BEAM_RANGE).Filter(f1_region).Filter(KS_PATHLENGTH_CUT, "pathlength cut").Filter(KS_MASS_CUT, "ks mass cut").Filter(P_P_CUT, "pp cut").Filter(delta_cut, "delta cut").Filter(lambda_cut, "lambda cut").Filter(missing_all_cut, "missing mass all").Filter(kstar_cut, "kstar cut")
    report = filtered_df.Report()
    return (filtered_df, report)

def apply_e_bin_filters_get_report(df, channel, e):
    if channel == 'pipkmks':
        delta_cut = PPIP_MASS_CUT
        lambda_cut = KMP_MASS_CUT
        missing_all_cut = MX2_PPIPKMKS_CUT
        kstar_cut = KSTAR_ALL_CUT_PIPKMKS
        f1_region = F1_SIGNAL_REGION_PIPKMKS
    elif channel == 'pimkpks':
        delta_cut = PPIM_MASS_CUT
        lambda_cut = KSP_MASS_CUT
        missing_all_cut = MX2_PPIMKPKS_CUT
        kstar_cut = KSTAR_ALL_CUT_PIMKPKS
        f1_region = F1_SIGNAL_REGION_PIMKPKS

    beam_cut = f'e_beam > {e - 0.5} && e_beam < {e + 0.5}'
    
    filtered_df = df.Filter(T_RANGE).Filter(beam_cut).Filter(f1_region).Filter(KS_PATHLENGTH_CUT, "pathlength cut").Filter(KS_MASS_CUT, "ks mass cut").Filter(P_P_CUT, "pp cut").Filter(delta_cut, "delta cut").Filter(lambda_cut, "lambda cut").Filter(missing_all_cut, "missing mass all").Filter(kstar_cut, "kstar cut")
    report = filtered_df.Report()
    return (filtered_df, report)


def compare_reports(report_pipkmks, report_pimkpks):
    cut_names = ['pathlength cut', 'ks mass cut', 'pp cut', 'delta cut', 'lambda cut', 'missing mass all', 'kstar cut']
    for cut_name in cut_names:
        cut_info1 = report_pipkmks.At(cut_name)
        cut_info2 = report_pimkpks.At(cut_name)
        print(f'Efficiency for {cut_name}: ')
        print(f'pipkmks: {cut_info1.GetEff()} || pimkpks: {cut_info2.GetEff()}')
        print("==" * 30)

def compare_binned_reports(report_7, report_8, report_9, report_10):
    cut_names = ['pathlength cut', 'ks mass cut', 'pp cut', 'delta cut', 'lambda cut', 'missing mass all', 'kstar cut']
    for cut_name in cut_names:
        cut_info7 = report_7.At(cut_name)
        cut_info8 = report_8.At(cut_name)
        cut_info9 = report_9.At(cut_name)
        cut_info10 = report_10.At(cut_name)
        print(f'Efficiency for {cut_name}: ')
        print(f'7 GeV: {cut_info7.GetEff()} || 8 GeV: {cut_info8.GetEff()} || 9 GeV: {cut_info9.GetEff()} || 10 GeV: {cut_info10.GetEff()}')
        print("==" * 50)

def plot_cut_per_e_vs_kkpi(df, channel, cut):
    if channel == 'pipkmks':
        delta_cut = PPIP_MASS_CUT
        lambda_cut = KMP_MASS_CUT
        missing_all_cut = MX2_PPIPKMKS_CUT
        kstar_cut = KSTAR_ALL_CUT_PIPKMKS
        f1_region = F1_SIGNAL_REGION_PIPKMKS
    elif channel == 'pimkpks':
        delta_cut = PPIM_MASS_CUT
        lambda_cut = KSP_MASS_CUT
        missing_all_cut = MX2_PPIMKPKS_CUT
        kstar_cut = KSTAR_ALL_CUT_PIMKPKS
        f1_region = F1_SIGNAL_REGION_PIMKPKS

    df = df.Filter(T_RANGE).Filter(BEAM_RANGE)
        


In [5]:
recon_df_pipkmks = build_recon_dataframe('pipkmks')
recon_df_pipkmks = define_columns(recon_df_pipkmks, 'pipkmks')

filepath: /work/halld/home/viducic/data/pipkmks/mc/phasespace/mc_pipkmks_phasespace_flat_bestX2_2018_spring.root
filepath: /work/halld/home/viducic/data/pipkmks/mc/phasespace/mc_pipkmks_phasespace_flat_bestX2_2018_fall.root
filepath: /work/halld/home/viducic/data/pipkmks/mc/phasespace/mc_pipkmks_phasespace_flat_bestX2_2017.root


In [6]:
recon_df_pimkpks = build_recon_dataframe('pimkpks')
recon_df_pimkpks = define_columns(recon_df_pimkpks, 'pimkpks')

filepath: /work/halld/home/viducic/data/pimkpks/mc/phasespace/mc_pimkpks_phasespace_flat_bestX2_2018_spring.root
filepath: /work/halld/home/viducic/data/pimkpks/mc/phasespace/mc_pimkpks_phasespace_flat_bestX2_2018_fall.root
filepath: /work/halld/home/viducic/data/pimkpks/mc/phasespace/mc_pimkpks_phasespace_flat_bestX2_2017.root


In [7]:
filtered_df_pipkmks, report_pipkmks = apply_filters_get_report(recon_df_pipkmks, 'pipkmks')
filtered_df_pimkpks, report_pimkpks = apply_filters_get_report(recon_df_pimkpks, 'pimkpks')

In [8]:
compare_reports(report_pipkmks, report_pimkpks)

Efficiency for pathlength cut: 
pipkmks: 72.31822204589844 || pimkpks: 72.39286804199219
Efficiency for ks mass cut: 
pipkmks: 93.08618927001953 || pimkpks: 93.12086486816406
Efficiency for pp cut: 
pipkmks: 89.7752914428711 || pimkpks: 89.77003479003906
Efficiency for delta cut: 
pipkmks: 90.08384704589844 || pimkpks: 46.61662292480469
Efficiency for lambda cut: 
pipkmks: 99.08928680419922 || pimkpks: 98.95333862304688
Efficiency for missing mass all: 
pipkmks: 93.55191040039062 || pimkpks: 93.93113708496094
Efficiency for kstar cut: 
pipkmks: 63.181217193603516 || pimkpks: 52.791839599609375


In [12]:
df_pipkmks_7, pipkmks_report_7 = apply_e_bin_filters_get_report(recon_df_pipkmks, 'pipkmks', 7)
df_pipkmks_8, pipkmks_report_8 = apply_e_bin_filters_get_report(recon_df_pipkmks, 'pipkmks', 8)
df_pipkmks_9, pipkmks_report_9 = apply_e_bin_filters_get_report(recon_df_pipkmks, 'pipkmks', 9)
df_pipkmks_10, pipkmks_report_10 = apply_e_bin_filters_get_report(recon_df_pipkmks, 'pipkmks', 10)

df_pimkpks_7, pimkpks_report_7 = apply_e_bin_filters_get_report(recon_df_pimkpks, 'pimkpks', 7)
df_pimkpks_8, pimkpks_report_8 = apply_e_bin_filters_get_report(recon_df_pimkpks, 'pimkpks', 8)
df_pimkpks_9, pimkpks_report_9 = apply_e_bin_filters_get_report(recon_df_pimkpks, 'pimkpks', 9)
df_pimkpks_10, pimkpks_report_10 = apply_e_bin_filters_get_report(recon_df_pimkpks, 'pimkpks', 10)

In [14]:
print('######   PI+ K- Ks   ######')
compare_binned_reports(pipkmks_report_7, pipkmks_report_8, pipkmks_report_9, pipkmks_report_10)
print('###########################')
print('###########################\n')
print('||||||   PI- K+ Ks   ||||||')
compare_binned_reports(pimkpks_report_7, pimkpks_report_8, pimkpks_report_9, pimkpks_report_10)

######   PI+ K- Ks   ######
Efficiency for pathlength cut: 
7 GeV: 69.55010223388672 || 8 GeV: 71.4935302734375 || 9 GeV: 72.70507049560547 || 10 GeV: 74.71925354003906
Efficiency for ks mass cut: 
7 GeV: 92.459228515625 || 8 GeV: 93.01036834716797 || 9 GeV: 93.2718276977539 || 10 GeV: 93.21273040771484
Efficiency for pp cut: 
7 GeV: 90.04171752929688 || 8 GeV: 89.70232391357422 || 9 GeV: 89.76628875732422 || 10 GeV: 89.80008697509766
Efficiency for delta cut: 
7 GeV: 83.01683807373047 || 8 GeV: 88.70043182373047 || 9 GeV: 91.26625061035156 || 10 GeV: 94.08120727539062
Efficiency for lambda cut: 
7 GeV: 95.468017578125 || 8 GeV: 98.89875030517578 || 9 GeV: 99.68699645996094 || 10 GeV: 99.97213745117188
Efficiency for missing mass all: 
7 GeV: 95.91722106933594 || 8 GeV: 94.66194152832031 || 9 GeV: 93.5036392211914 || 10 GeV: 90.69915008544922
Efficiency for kstar cut: 
7 GeV: 63.81215286254883 || 8 GeV: 63.58944320678711 || 9 GeV: 63.13996887207031 || 10 GeV: 62.225711822509766
|||||| 